In [13]:
import ROOT as r
import json

from pathlib import Path
import math

In [2]:
dataset_runnable = json.loads(Path("dataset_runnable/af_v3_marcus.json").read_text())
metadata=dataset_runnable['N2_220_N1_200_HH']['metadata']

kFactor = metadata["kFactor"]
sumOfWeights = metadata["sum_of_weights"]
genFiltEff = metadata["genFiltEff"]
xs = metadata["crossSection"]
lum = metadata["luminosity"]
weight_norm_mc = xs*genFiltEff*kFactor*lum/sumOfWeights

files_to_open=["/data/mhance/SUSY/ntuples/v3/output_N2_220_N1_200_HH.root"]

In [3]:
h_baseline_pt=r.TH1F("baseline_pt","baseline_pt",10000,0,10000)

h_istruth_it_ii_pt=r.TH1F("istruth_istight_isiso_pt","istight_isiso",10000,0,10000)
h_istruth_it_ni_pt=r.TH1F("istruth_istight_noiso_pt","istight_noiso",10000,0,10000)
h_istruth_nt_ii_pt=r.TH1F("istruth_notight_isiso_pt","notight_isiso",10000,0,10000)
h_istruth_nt_ni_pt=r.TH1F("istruth_notight_noiso_pt","notight_noiso",10000,0,10000)

h_notruth_it_ii_pt=r.TH1F("notruth_istight_isiso_pt","istight_isiso",10000,0,10000)
h_notruth_it_ni_pt=r.TH1F("notruth_istight_noiso_pt","istight_noiso",10000,0,10000)
h_notruth_nt_ii_pt=r.TH1F("notruth_notight_isiso_pt","notight_isiso",10000,0,10000)
h_notruth_nt_ni_pt=r.TH1F("notruth_notight_noiso_pt","notight_noiso",10000,0,10000)

In [25]:
eventcount=0
premindphicut=0
prejetscuts=0
postleadjetptcut=0
postmindphicut=0
postphotonpresel=0

postmetcut=0
postphtight=0
postlepveto=0

cutflowevents=0
cutflowsumw=0

filesopened=0
for file_to_open in files_to_open:
    filesopened+=1
    print(f"Processing file {file_to_open}, {filesopened} of {len(files_to_open)}")
    f=r.TFile(file_to_open,"RO")
    t=f.Get("picontuple")
    totalevents=t.GetEntriesFast()
    for e in t:
    
        eventcount+=1
        if eventcount%50000 == 0:
            print(f"Processed {eventcount:6d}/{totalevents} events")
        
        #=================================================================
        # preselection
        #
        met=e.met_met
        met_phi=e.met_phi
        met_tlv=r.TLorentzVector()
        met_tlv.SetPtEtaPhiM(met,0,met_phi,0)
    
        # MET requirement, for now just look at low MET
        if met<200000.: 
            continue

        postmetcut+=1   

        photon_tlv = r.TLorentzVector()
        photon_tlv.SetPtEtaPhiM(e.ph_pt,
                                e.ph_eta,
                                e.ph_phi,
                                0)
        
        postphtight+=1
        prejetscuts+=1
    
        if e.j1_pt<150000.:
            continue
    
        postleadjetptcut+=1

        if e.nBTagJets>0:
            continue
    
        premindphicut+=1
        
        if abs(e.mindPhiJetMet)<0.4: 
            continue
    
        postmindphicut+=1            

        if (e.nElectrons + e.nMuons)>0:
            continue
        
        postlepveto+=1

        mT_A = r.TMath.Sqrt(2*photon_tlv.Pt()*met*(1.-r.TMath.Cos(photon_tlv.DeltaPhi(met_tlv))))

        #print(mT_A)
        if mT_A > 50000.:
            continue

        ## overlap removal for vgamma samples
        #if not e.event_select_vgammaOverlap7: 
        #    continue
        
        h_baseline_pt.Fill(e.ph_pt/1000.)
    
        #=================================================================
    
        #=================================================================
        # now we have the index of the photon to keep.  Now figure out if this is a truth-matched photon or not.
        # will need to fix this whenever Giordon et al decide on a truth matching scheme.
        #print(f"{e.ph_truthType[photon_index]} {e.ph_truthOrigin[photon_index]} {e.ph_truthpdgId[photon_index]}")
        ph_truthmatch=((e.ph_truthType!=16) and (e.ph_truthType!=0))
    
        # the tight and isolated flags are easier:
        ph_tight=(ord(e.ph_select_tightID)>0)
        ph_iso  =(ord(e.ph_select_tightIso)>0)
    
        if ph_truthmatch:
            if       ph_tight and     ph_iso: h_istruth_it_ii_pt.Fill(e.ph_pt/1000.)
            elif     ph_tight and not ph_iso: h_istruth_it_ni_pt.Fill(e.ph_pt/1000.)
            elif not ph_tight and     ph_iso: h_istruth_nt_ii_pt.Fill(e.ph_pt/1000.)
            elif not ph_tight and not ph_iso: h_istruth_nt_ni_pt.Fill(e.ph_pt/1000.)
        else:
            if       ph_tight and     ph_iso: h_notruth_it_ii_pt.Fill(e.ph_pt/1000.)
            elif     ph_tight and not ph_iso: h_notruth_it_ni_pt.Fill(e.ph_pt/1000.)
            elif not ph_tight and     ph_iso: h_notruth_nt_ii_pt.Fill(e.ph_pt/1000.)
            elif not ph_tight and not ph_iso: h_notruth_nt_ni_pt.Fill(e.ph_pt/1000.)

        jetSF = e.weight_fjvt_effSF * e.weight_ftag_effSF_GN2v01_Continuous * e.weight_jvt_effSF 
        photonSF = e.ph_id_effSF_tightID * e.ph_id_effSF_tightIso
        SF = jetSF * photonSF

        
        if ph_tight and ph_iso:    
            cutflowevents += 1.
            cutflowsumw += (e.weight_mc*e.weight_pileup*weight_norm_mc*SF)
            #print(e.eventNumber)
            #print(f"{weight_norm_mc:.3f} {e.weight_mc:.3f} {e.weight_pileup:.3f} {SF:.3f} {(e.weight_mc*e.weight_pileup*weight_norm_mc*SF):.3f}")
        #=================================================================

Processing file /data/mhance/SUSY/ntuples/v3/output_N2_220_N1_200_HH.root, 1 of 1


In [26]:
print(f"total events  {eventcount}")
print(f"post met      {postmetcut}")
print(f"post phtight  {postphtight}")
print(f"pre jets cuts {prejetscuts}") 

print(f"post leadjet  {postleadjetptcut}")
print(f"post btagveto {premindphicut}")  
print(f"post mindphi  {postmindphicut}")
print(f"post lepveto  {postlepveto}")
print(f"post photonpr {postphotonpresel}")
print(f"baseline:     {h_baseline_pt.GetSumOfWeights()}")
print(f"cutflow events:  {cutflowevents}")
print(f"cutflow sumw:    {cutflowsumw}")

print(f"A:  truth,  tight, !isolated: {h_istruth_it_ni_pt.GetSumOfWeights()}")
print(f"B:  truth, !tight, !isolated: {h_istruth_nt_ni_pt.GetSumOfWeights()}")
print(f"C:  truth,  tight,  isolated: {h_istruth_it_ii_pt.GetSumOfWeights()}")
print(f"D:  truth, !tight,  isolated: {h_istruth_nt_ii_pt.GetSumOfWeights()}")

print(f"A: !truth,  tight, !isolated: {h_notruth_it_ni_pt.GetSumOfWeights()}")
print(f"B: !truth, !tight, !isolated: {h_notruth_nt_ni_pt.GetSumOfWeights()}")
print(f"C: !truth,  tight,  isolated: {h_notruth_it_ii_pt.GetSumOfWeights()}")
print(f"D: !truth, !tight,  isolated: {h_notruth_nt_ii_pt.GetSumOfWeights()}")

total events  8741
post met      8741
post phtight  8741
pre jets cuts 8741
post leadjet  8134
post btagveto 7509
post mindphi  6654
post lepveto  5819
post photonpr 0
baseline:     21993.0
cutflow events:  1303.0
cutflow sumw:    232.9457529147704
A:  truth,  tight, !isolated: 3724.0
B:  truth, !tight, !isolated: 924.0
C:  truth,  tight,  isolated: 14786.0
D:  truth, !tight,  isolated: 2110.0
A: !truth,  tight, !isolated: 71.0
B: !truth, !tight, !isolated: 193.0
C: !truth,  tight,  isolated: 110.0
D: !truth, !tight,  isolated: 75.0
